# Notebook to get dice scores for components

Uses the yeo atlas and the anti-cole atlas to calculate dice scores to get grey matter defintions

In [ ]:
import nibabel as nb
import numpy as np
from conilab.data_functions.image_handling import decompose_cifti, downsample
from conilab.data_functions.data import save_json
from nfact_id.functions.component_def_functions import define_components_by_dice
import nilearn.image as nimg
from decouple import config
import pandas as pd
import os

In [27]:
base_dir = config("base_dir")
atlas_locations = os.path.join(base_dir, "parcellations")
json_location = os.path.join(base_dir, "component_defintion", 'jsons_to_build_df')

## Get high and low res surfaces + low res subcortical volume to downsample atlases to

In [18]:
l_high_res_surface = nb.load(os.path.join(atlas_locations, "S1200.L.inflated_MSMAll.32k_fs_LR.surf.gii")).darrays[0].data
l_low_res_surface = nb.load(os.path.join(base_dir, "viewing_surfaces", "L_inflated_10k.surf.gii")).darrays[0].data
r_high_res_surface = nb.load(os.path.join(atlas_locations, "S1200.R.inflated_MSMAll.32k_fs_LR.surf.gii")).darrays[0].data
r_low_res_surface = nb.load(os.path.join(base_dir, "viewing_surfaces", "R_inflated_10k.surf.gii")).darrays[0].data
subcortcal = nb.load(os.path.join(atlas_locations, "combined_rois.nii.gz"))

## Load YEO and COLE atlas

In [19]:
cole_atlas = nb.load(os.path.join(atlas_locations, "CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_netassignments_LR.dlabel.nii"))
yeo_atlas = nb.load(os.path.join(atlas_locations, "yeo_17.32k_fs_LR.dlabel.nii"))

## Load NFACT components

In [20]:
gmf = nb.load(os.path.join(base_dir, "nfact_decomp_mh", "components", "NMF", "decomp", "G_NMF_dim100.dscalar.nii" ))
nmf_comp = decompose_cifti(gmf)

## Decompose and downsample ciftis

### Yeo atlas

This is actually 3 atlases together and has been turned from a cifit 1 to cifit 2 in workbench. 
So we need the 1st part of the array which the 17

In [21]:
yeo = decompose_cifti(yeo_atlas)
yeo_left_atlas = downsample(yeo['L_surf'], l_low_res_surface, l_high_res_surface)
yeo_right_atlas = downsample(yeo['R_surf'], r_low_res_surface, r_high_res_surface)
yeo_atlas = np.concat([yeo_left_atlas.squeeze(), yeo_right_atlas.squeeze()])[:, 1]

In [22]:
yeo_label_df = pd.read_csv(os.path.join(atlas_locations, "RSN_17.csv"))
yeo_label_df['Network Name'] = yeo_label_df['Network Name'].apply(lambda x: x.lstrip())

### Cole atlas

This atlas has a subcortical element so this also needs downsampling

In [23]:
cole = decompose_cifti(cole_atlas)
l_surf_cole = downsample(cole['L_surf'], l_low_res_surface, l_high_res_surface) 
r_surf_cole = downsample(cole['R_surf'], r_low_res_surface, r_high_res_surface) 
data = cole['vol'].get_fdata()
cole_subcortical = nb.Nifti1Image(data[:,:,:,0], affine=cole['vol'].affine, header=cole['vol'].header)
subcortical_cole = nimg.resample_to_img(cole_subcortical, subcortcal, copy_header=True, force_resample=True, interpolation="nearest")
cole_atlas_data = np.concat([l_surf_cole.squeeze(), r_surf_cole.squeeze(), subcortical_cole.get_fdata().flatten()])

In [24]:
cole_label_df = pd.read_csv(os.path.join(atlas_locations, "ColeAnticevic_names.csv"))

## Get DICE scores from atlas

In [ ]:
cole_dice = define_components_by_dice(cole_label_df['Network Name'].to_list()[1:], nmf_comp, nmf_comp['L_surf'].shape[1], cole_label_df, cole_atlas_data)
yeo_dice = define_components_by_dice(yeo_label_df['Network Name'].to_list()[1:], nmf_comp, nmf_comp['L_surf'].shape[1], yeo_label_df, yeo_atlas, subcortical=False)

## Save jsons

In [ ]:
save_json(os.pth.join(json_location, "cole_dice_scores.json"), cole_dice)
save_json(os.pth.join(json_location, "yeo_17_dice_scores.json"), yeo_dice)